In [52]:
import sqlite3
import pandas as pd

conn = sqlite3.connect('ecommerce.db')

df = pd.read_sql_query("""
    SELECT user_id, order_date
    FROM orders
""", conn)

df


,user_id,order_date
0,24,2025-05-18 02:48:50.400602
1,49,2025-04-19 04:42:09.342558
2,41,2025-04-29 12:58:35.088508
3,25,2025-08-28 01:58:48.148105
4,50,2025-07-07 03:08:42.311355
...,...,...
195,5,2025-07-19 09:34:50.885877
196,33,2025-04-24 10:45:19.935082
197,13,2025-05-22 17:25:37.489298
198,7,2025-07-11 09:20:02.347473


## 1. **User Cohorts & Retention**

- **Query**: Calculate how many users sign up each month and how many of them place an order in subsequent months (retention).

In [60]:
display(
    pd.read_sql_query(
        '''
    SELECT user_id, strftime('%Y-%m', order_date) AS order_month
    FROM orders
    WHERE user_id IS '24'
    ORDER BY order_month
    '''
    , conn), 
)

,user_id,order_month
0,24,2025-03
1,24,2025-04
2,24,2025-04
3,24,2025-05


In [78]:
df = pd.read_sql_query("""
WITH new_users AS (
    SELECT 
        strftime('%Y-%m', signup_date) AS signup_month,
        COUNT(user_id) AS total_new_users
    FROM users
    GROUP BY signup_month
), 
-- We calculate the number of new users per month
user_orders AS (
    SELECT 
        u.user_id,
        strftime('%Y-%m', u.signup_date) AS signup_month,
        strftime('%Y-%m', o.order_date) AS order_month
    FROM users u
    JOIN orders o
    ON u.user_id = o.user_id
),
-- We obtain the list of users who made an order, their signup month and the months they made an order 
retention AS (
    SELECT 
        signup_month,
        order_month,
        COUNT(DISTINCT user_id) AS retained_users
    FROM user_orders
    GROUP BY signup_month, order_month
)
-- We group the distinct users by signup month and order month, calculating the # ofusers who
-- made an order after signing up
SELECT
    r.signup_month,
    n.total_new_users,
    r.retained_users as num_users_who_ordered_in_following_months,
    ROUND(CAST(r.retained_users AS REAL) / n.total_new_users, 2) AS retention_rate
FROM retention r
JOIN new_users n
ON r.signup_month = n.signup_month
ORDER BY r.signup_month;
-- And finally we divide the number of total users who signed up by the num of users who made
-- an oder after signing up, grouped by signup month and order month
""", conn)

df

,signup_month,total_new_users,num_users_who_ordered_in_following_months,retention_rate
0,2024-09,5,2,0.4
1,2024-09,5,1,0.2
2,2024-09,5,2,0.4
3,2024-09,5,2,0.4
4,2024-09,5,3,0.6
...,...,...,...,...
67,2025-08,2,1,0.5
68,2025-08,2,1,0.5
69,2025-08,2,1,0.5
70,2025-08,2,2,1.0


## 2. **Repeat Orders & Time Gaps**

- **Query**: Find the average number of days between orders per user.
- **Hint**:
    - Sort orders per user by `order_date`.
    - Use `LAG(order_date)` and compute the difference.
    - Then take `AVG()` per `user_id`.

In [ ]:
df = pd.read_sql_query("""
WITH orders_with_lag AS (
    SELECT
        user_id,
        order_date,
        LAG(order_date) OVER (PARTITION BY user_id ORDER BY order_date) AS prev_order_date
    FROM orders
)
SELECT
    user_id,
    order_date,
    (strftime('%s', order_date) - strftime('%s', prev_order_date)) / 86400 AS day_diff,
    ROUND(AVG((strftime('%s', order_date) - strftime('%s', prev_order_date)) / 86400)
        OVER (PARTITION BY user_id), 0) AS avg_day_diff
FROM orders_with_lag;


""", conn)

df

,user_id,order_date,day_diff,avg_day_diff
0,1,2025-03-28 20:21:39.178127,NaN,46.0
1,1,2025-05-23 16:07:34.629729,55.0,46.0
2,1,2025-05-27 15:35:17.246004,3.0,46.0
3,1,2025-08-16 12:32:34.654958,80.0,46.0
4,2,2025-07-21 17:27:35.961724,NaN,5.0
...,...,...,...,...
195,50,2025-07-03 10:11:56.840566,NaN,11.0
196,50,2025-07-04 01:59:49.191465,0.0,11.0
197,50,2025-07-07 03:08:42.311355,3.0,11.0
198,50,2025-07-25 02:53:11.915015,17.0,11.0
